<a href="https://colab.research.google.com/github/EmpyEmpt/titanic/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sklearn
!pip install tensorflow_decision_forests
%tensorflow_version 2.x  

Жирнющий импорт потому что я много что тестировал и все импорты складывал в одно место.
Удалять ничего не буду (!!!) ибо вдруг мне захочется еще что-то попробовать, а колаб не сильно расстроится из-за этого

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow as tf
import tensorflow_decision_forests as tfdf

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

ДА я в наглую скопировал всю трансформацию датасета, НО в свое оправдание хочу сказать, что я игрался с другим форматированием/отключения части трансформаций/использовал встроенный в TF трансформер данных и т.п., но зачастую результаты:
* Становились хуже
* Не менялись

А никаких прямых улучшений я придумать не смог


In [ ]:
train_df = train_df.drop(['Ticket', 'Cabin', 'PassengerId'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)

train_df['Title'] = train_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
test_df['Title'] = test_df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

for dataset in [train_df, test_df]:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in [train_df, test_df]:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0) 

train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)

train_df['Sex'] = train_df['Sex'].map({'male': 0, 'female': 1}).astype(int)
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1}).astype(int)

guess_ages = np.zeros((2,3))
for dataset in [train_df, test_df]:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median()
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

for dataset in [train_df, test_df]:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4

train_df = train_df.drop(['AgeBand'], axis=1)
train_df['FamilySize'] = train_df['SibSp'] + train_df['Parch'] + 1 
test_df['FamilySize'] = test_df['SibSp'] + test_df['Parch'] + 1

train_df = train_df.drop(['Parch', 'SibSp'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp'], axis=1)

freq_port = train_df.Embarked.dropna().mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(freq_port)
test_df['Embarked'] = test_df['Embarked'].fillna(freq_port)

train_df['Embarked'] = train_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_df['Embarked'] = test_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
for dataset in [train_df, test_df]:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)

In [ ]:
def split_dataset(dataset, test_ratio=0.20):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

In [ ]:
# train_ds_pd, test_ds_pd = split_dataset(train_df)
# train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='Survived')
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label='Survived')

# keras нужен Dataset, у которого указан target-label
cmpl_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label='Survived')

/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)


In [ ]:
# train_, test_ = split_dataset(train_df)
# trainY = train_['Survived']
# trainX = train_.drop(columns = ['Survived'])
# testY = test_['Survived']
# testX = test_.drop(columns = ['Survived'])

In [ ]:
cmpl_df = train_df.copy()
cmpl_df_y = train_df['Survived']
cmpl_df_x = train_df.drop(columns = ['Survived'])
cmpl_df_x = tf.convert_to_tensor(cmpl_df_x, dtype=tf.int32)
cmpl_df_y = tf.convert_to_tensor(cmpl_df_y, dtype=tf.int32)

In [ ]:
# trainX_ = tf.convert_to_tensor(trainX, dtype=tf.int32)
# trainY_ = tf.convert_to_tensor(trainY, dtype=tf.int32)

# testX_ = tf.convert_to_tensor(testX, dtype=tf.int32)
# testY_ = tf.convert_to_tensor(testY, dtype=tf.int32)

In [ ]:
# testX_.shape

TensorShape([187, 7])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [ ]:
def build():
  model = Sequential()
  input_shape = (7,)
  model.add(Dense(49, activation = 'relu'))
  model.add(Dense(147, activation = 'relu'))
  model.add(Dropout(0.25))
  model.add(Dense(49, activation = 'relu'))
  model.add(Activation("softmax"))
  model.add(Dense(1))
  return model

In [ ]:
def get_model():
    model = build()
    model.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError(), metrics=['accuracy'])

    # model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
    return model

In [ ]:
model = get_model()

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='training_1/cp-{epoch:04d}.ckpt',
    verbose=1,
    save_best_only=True,
    save_weights_only=True)

model.fit(cmpl_df_x, 
          cmpl_df_y,
          epochs=1,
          # validation_data=(testX_, testY_),
          shuffle=True,
          callbacks=[cp_callback])
loss, acc = model.evaluate(cmpl_df_x, cmpl_df_y, verbose=2)

16/28 [================>.............] - ETA: 0s - loss: 0.1700 - accuracy: 0.8301WARNING:tensorflow:Can save best model only with val_loss available, skipping.


28/28 [==============================] - 0s 4ms/step - loss: 0.1784 - accuracy: 0.8215
28/28 - 0s - loss: 0.1727 - accuracy: 0.8283 - 56ms/epoch - 2ms/step


In [ ]:
model.save("md.h5")
loss, acc = model.evaluate(cmpl_df_x, cmpl_df_y, verbose=2)

28/28 - 0s - loss: 0.1727 - accuracy: 0.8283 - 89ms/epoch - 3ms/step


In [ ]:
# model = tf.keras.models.load_model('md.h5')
# loss, acc = model.evaluate(cmpl_df_x, cmpl_df_y, verbose=2)

28/28 - 0s - loss: 0.1727 - accuracy: 0.8283 - 297ms/epoch - 11ms/step


In [ ]:
test = test_df.drop(columns = ['PassengerId'])

In [ ]:
test = tf.convert_to_tensor(test, dtype=tf.int32)

In [ ]:
model = tf.keras.models.load_model('md3.h5')

In [ ]:
# test = tfdf.keras.pd_dataframe_to_tf_dataset(test_df)
pred = model.predict(test)
pred_list = []
for i in pred:
  if i >= 0.5:
    pred_list.append(1)
  else:
    pred_list.append(0)

submission = test_df['PassengerId'].to_frame()
submission['Survived'] = pred_list
submission.to_csv('model3.csv', index = False)

С гиперпараметрами тоже пытался играться, этот template показывает себя *мизерно* лучше дефолтной настройки, но а почему бы не дожать последнюю каплю?
*(acc. 0.8687 vs acc. 0.8642)*


In [ ]:
# MODEL 1
model = tfdf.keras.RandomForestModel(hyperparameter_template = 'benchmark_rank1')
# model = tfdf.keras.RandomForestModel()


model.compile(metrics=['accuracy'])
model.fit(cmpl_ds)

TypeError: ignored

In [ ]:
evaluation = model.evaluate(cmpl_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 0.8687

loss: 0.0000
accuracy: 0.8687


In [ ]:
test = tfdf.keras.pd_dataframe_to_tf_dataset(test_df)
pred = model.predict(test)

In [ ]:
pred_list = []
for i in pred:
  if i >= 0.5:
    pred_list.append(1)
  else:
    pred_list.append(0)
len(pred_list)

418

In [ ]:
submission = test_df['PassengerId'].to_frame()
submission['Survived'] = pred_list
submission.to_csv('submission.csv', index = False)

In [ ]:
model.save("project/model")

RandomForestClassifier просто показывает себя лучше, ничего я тут не сделаю... но и результат его сабмитить не собираюсь

In [ ]:
# MODEL 2

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 7), (891,), (418, 7))

In [ ]:
rf = RandomForestClassifier()

rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)
acc_log = round(rf.score(X_train, Y_train) * 100, 2)
acc_log

88.55

Из трех моделей CatBoostClassifier больше всех нюхает бебру, но было интересно на него посмотреть

Проще чем TF, сложнее чем sklearn, может решусь еще fastai опробовать...

In [ ]:
# CatBoost

In [ ]:
!pip install catboost
!pip install scikit-learn
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

In [ ]:
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

In [ ]:
categorical_features_indices = np.where(X.dtypes != float)[0]

In [ ]:
modelCat = CatBoostClassifier(
    custom_loss=[metrics.Accuracy()],
    # random_seed=42,
    logging_level='Silent'
)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.75, random_state=42)

X_test = test_df

In [ ]:
cv_params = modelCat.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
) 

modelCat.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=(X_validation, y_validation),
    plot=True
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']),
    cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
    np.argmax(cv_data['test-Accuracy-mean'])
))

Best validation accuracy score: 0.83±0.02 on step 196


In [ ]:
print(f'Precise validation accuracy score: {np.max(cv_data["test-Accuracy-mean"])}')

Precise validation accuracy score: 0.8294051627384961
